In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
from sklearn import svm
import csv
import pandas as pd

In [2]:
#Load data
data = pd.read_csv('labeled_data.csv', encoding = "ISO-8859-1", low_memory=False)
data.head(5)

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [3]:


#Make 90% of data for training
#Original dataset said 0=Hate speech, 1=Offensive but not hate speech, 2=neither 
#We need binary labels so first we made +1 only if it is not hate speech (>0) but this made majority of the dataset +1 and very few -1. 
#So, changed labels so only classified as non-hate if orig label (>1)
training = open("training_data.txt", "w+")
for i in range(int(0.9*data.shape[0])):
  tweet = data["tweet"][i].replace("\n", " ")
  label = data["class"][i]
  if label > 1:
    label = "+1"
  else:
    label = "-1"
  
  training.write(label + " " + tweet + "\n")

training.close()

validation = open("validation_data.txt", "w+")
for i in range(1239):
  tweet = data["tweet"][22304+i].replace("\n", " ")

  label = data["class"][22304+i]
  if label > 1:
    label = "+1"
  else:
    label = "-1"
  
  validation.write(label + " " + tweet + "\n")

validation.close()

testing = open("testing_data.txt", "w+")
for i in range(1240):
  tweet = data["tweet"][23543+i].replace("\n", " ")

  label = data["class"][23543+i]
  if label > 1:
    label = "+1"
  else:
    label = "-1"
  
  testing.write(label + " " + tweet + "\n")

testing.close()



In [4]:
training_tweets = []
training_labels = []
train = open("training_data.txt", "r")
train_lines = train.readlines()
train_lines = [line.strip() for line in train_lines]
for line in train_lines:
  tweet = line[3:]
  training_tweets.append(tweet)
  training_labels.append(int(line[:2]))

training_tweets = np.array(training_tweets)
training_labels = np.array(training_labels)

print(training_tweets[:5])
print(training_labels[:5])

["!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out..."
 '!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!'
 '!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit'
 '!!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny'
 '!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;']
[ 1 -1 -1 -1 -1]


In [5]:
test_tweets = []
test_labels = []
test = open("testing_data.txt", "r")
test_lines = test.readlines()
test_lines = [line.strip() for line in test_lines]
for line in test_lines:
  tweet = line[3:]
  test_tweets.append(tweet)
  test_labels.append(int(line[:2]))

test_tweets = np.array(test_tweets)
test_labels = np.array(test_labels)

print(test_tweets[:5])
print(test_labels[:5])

["ain't no bitch got shit on me" "ain't nuttin ta cut that bitch off"
 "ain't the bitch Mimi still wit Gay ass Nikko????"
 "ain't trusting no bitch or no nigga... hoe or no sucka"
 'aint nuthin retarded bout Gucci but this gold rolex']
[-1 -1 -1 -1 -1]


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Create feature vectors
vectorizer = TfidfVectorizer(min_df = 10,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
train_vectors = vectorizer.fit_transform(training_tweets)
print(vectorizer.vocabulary_)
test_vectors = vectorizer.transform(test_tweets)

{'rt': 1809, 'as': 240, 'woman': 2399, 'you': 2450, 'shouldn': 1905, 'complain': 552, 'about': 146, 'up': 2275, 'your': 2452, 'house': 1105, 'amp': 207, 'man': 1387, 'should': 1903, 'always': 199, 'take': 2079, 'the': 2134, 'trash': 2216, 'out': 1600, 'boy': 382, 'dats': 631, 'cold': 538, 'tyga': 2259, 'bad': 276, 'for': 883, 'dat': 628, 'hoe': 1077, 'in': 1147, '1st': 103, 'place': 1659, 'dawg': 634, '80sbaby4life': 124, 'ever': 768, 'fuck': 911, 'bitch': 341, 'and': 209, 'she': 1887, 'start': 2021, 'to': 2190, 'cry': 599, 'be': 298, 'confused': 555, 'shit': 1892, 'look': 1343, 'like': 1305, 'tranny': 2214, 'hear': 1044, 'me': 1405, 'might': 1429, 'true': 2228, 'or': 1589, 'it': 1170, 'than': 2128, 'who': 2370, 'told': 2194, 'ya': 2431, 'just': 1209, 'claim': 517, 'so': 1964, 'faithful': 803, 'down': 705, 'somebody': 1974, 'but': 433, 'still': 2039, 'fucking': 915, 'with': 2393, 'hoes': 1078, '128514': 53, 'can': 453, 'not': 1548, 'sit': 1927, 'hate': 1029, 'on': 1580, 'another': 215,

In [7]:
import time
from sklearn import svm
from sklearn.metrics import classification_report
# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(train_vectors, training_labels)
t1 = time.time()
prediction_linear = classifier_linear.predict(train_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1
# results
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = classification_report(training_labels, prediction_linear, output_dict=True)
print("Accuracy: " + str(report["accuracy"]))

Training time: 16.077673s; Prediction time: 10.961193s
Accuracy: 0.9684361549497847


In [8]:
from sklearn.svm import LinearSVC
X_train=train_vectors
y_train=training_labels
X_test=test_vectors #should be around 30% of you your data
y_test=test_labels
svm = LinearSVC()
svm.fit(X_train, y_train)
count = 0
false_negative = 0
false_positive = 0
for item, label in zip(X_test, y_test):
  count += 1
  result = svm.predict(item)
  if result != label:
      print(test_tweets[count - 1], result, label)
      if result == 1:
        false_positive += 1
      else:
        false_negative += 1
print(false_positive)
print(false_negative)

and another one &#8220;@07Cannon: Two hoes? RT "@xoxo_imaunicorn: y'all know what we are?! &#128514;&#128514;" http://t.co/XI5Aod1FAo&#8221; [-1] 1
apparently I'm pathetic trash alright (-: [-1] 1
auto tranny [-1] 1
basement diving mook bastards [1] -1
butt chug 4lokos or nah [1] -1
chill zahra im getting crackers rn [1] -1
country boy, hick lyfe, white tee, rebel flag [-1] 1
cut up porkchop, canned pinto beans, a small can of carrots, can of corn, celery, a potato. cayenne powder + crushed red pepper [-1] 1
f u tumblr bc my blog is black &amp; white but there's so many great colored things I could reblog fml [-1] 1
flappy bird makes me want to smash my fuckin screen [1] -1
geekylesbianscientists: theblairbetch: douglasrichardsonships: kayla-bird: okay but consider this though:... http://t.co/zAHyXtaVvo [1] -1
guess I will mess with it later! not sure what else to do! maybe trash it! lol!!! not yet maybe after I when the lottery! [-1] 1
guys that cheat on their girlfriend are trash.  t

In [9]:
#Gaussian Radial Basis Function (RBF)
import time
from sklearn import svm
from sklearn.metrics import classification_report
rbf_clf = svm.SVC(kernel='rbf', C=1.0)
t0 = time.time()
rbf_clf.fit(train_vectors, training_labels)
t1 = time.time()
prediction_rbf = rbf_clf.predict(train_vectors)
t2 = time.time()
time_rbf_train = t1-t0
time_rbf_predict = t2-t1
# results
print("Training time: %fs; Prediction time: %fs" % (time_rbf_train, time_rbf_predict))
report = classification_report(training_labels, prediction_rbf, output_dict=True)
print("Accuracy: " + str(report["accuracy"]))


Training time: 77.380319s; Prediction time: 22.318357s
Accuracy: 0.989105093256815


In [10]:
from sklearn import svm
from sklearn.svm import SVC
X_train=train_vectors
y_train=training_labels
X_test=test_vectors #should be around 30% of you your data
y_test=test_labels
svm = svm.SVC(kernel='rbf', C=1.0)
svm.fit(X_train, y_train)
count = 0
for item, label in zip(X_test, y_test):
  count += 1
  result = svm.predict(item)
  if result != label:
      print(test_tweets[count - 1], result, label)
      if result == 1:
        false_positive += 1
      else:
        false_negative += 1
print(false_positive)
print(false_negative)

and another one &#8220;@07Cannon: Two hoes? RT "@xoxo_imaunicorn: y'all know what we are?! &#128514;&#128514;" http://t.co/XI5Aod1FAo&#8221; [-1] 1
animal crackers are the fuckin bomb, yo [1] -1
auto tranny [-1] 1
b/c she likes black men and rap music? Huh lol RT @Magssca: @WaymoTheGod in general she wants to be colored [-1] 1
basement diving mook bastards [1] -1
butt chug 4lokos or nah [1] -1
chill zahra im getting crackers rn [1] -1
coon RT @HumbleTeej: &lt;fills coffee up halfway&gt; &lt;uses 11 Irish cremes to fill other half&gt; [-1] 1
country boy, hick lyfe, white tee, rebel flag [-1] 1
damn they gonna leave beth here long term. trash [1] -1
f u tumblr bc my blog is black &amp; white but there's so many great colored things I could reblog fml [-1] 1
flappy bird makes me want to smash my fuckin screen [1] -1
geekylesbianscientists: theblairbetch: douglasrichardsonships: kayla-bird: okay but consider this though:... http://t.co/zAHyXtaVvo [1] -1
guess rubes doesn't want to go on a 